In [1]:
import numpy as np
from scipy.spatial import KDTree
from collections import deque

G_NORMAL = 0.05
G_MICRO = 0

class Particle:
    def __init__(self, unique_id, model, base_type=None):
        self.unique_id = unique_id
        self.model = model
        self.base_type = base_type
        self.mass = {'A': 1.0, 'G': 1.2, 'C': 0.8, 'U': 0.85, None: 2.0}[base_type]
        self.resources = 0
        self.pos = np.array([np.random.uniform(0, 99.999), np.random.uniform(0, 99.999)])
        self.bonds = set()
        self.is_cell = False
        self.radius = 0
        self.cluster_id = None

    def step(self):
        density = 1025 + (1076 - 1025) * (100 - self.pos[1]) / 100
        gravity_factor = 1.5 if self.cluster_id is not None else 1
        if self.model.G != G_MICRO:
            self.pos[1] += -self.model.G * self.mass * gravity_factor
        step_size = 0.5 / np.sqrt(density / 1025) / (2 if self.is_cell else 1)
        self.pos += np.random.uniform(-step_size, step_size, 2)
        self.pos = np.clip(self.pos, 0, 99.999)
        base_density = max(0, 10 - self.pos[1] / 10) if self.model.G != G_MICRO else 5
        uptake = base_density * 0.1 * (density / 1025) * (2 if self.is_cell else 1)
        if self.cluster_id is not None:
            cluster_size = len([p for p in self.model.particles if p.cluster_id == self.cluster_id])
            uptake *= (1 + 0.1 * cluster_size)
        self.resources = min(self.resources + uptake, 50)
        if self.is_cell:
            self.resources = max(self.resources - 1, 0)  # Metabolism cost
            if self.resources < 5:
                self.is_cell = False
                self.radius = 0
                self.mass = 1.0  # Revert to avg nucleotide mass

class LifeModel:
    def __init__(self, N_particles, G):
        self.G = G
        self.particles = [Particle(i, self, ['A', 'G', 'C', 'U'][i % 4]) for i in range(N_particles)]
        self.bond_events = 0
        self.removed_particles = set()
        self.next_cluster_id = 0

    def step(self):
        for p in self.particles:
            p.step()
        positions = np.array([p.pos for p in self.particles])
        if len(positions) == 0:
            return
        
        tree = KDTree(positions)
        for i, particle in enumerate(self.particles):
            if particle.unique_id in self.removed_particles:
                continue
            indices = tree.query_ball_point(particle.pos, 1.0 if not particle.is_cell else 2.0)
            for j in indices:
                if i == j or self.particles[j].unique_id in self.removed_particles:
                    continue
                other = self.particles[j]
                bond_prob = 1.0 if particle.pos[1] < 10 else 0.8  # Quantum uncertainty
                if not particle.is_cell and not other.is_cell:
                    can_bond = (particle.base_type == 'A' and other.base_type == 'U') or \
                               (particle.base_type == 'U' and other.base_type == 'A') or \
                               (particle.base_type == 'G' and other.base_type == 'C') or \
                               (particle.base_type == 'C' and other.base_type == 'G')
                    bond_threshold = 4 if particle.pos[1] < 10 else 5
                    if (can_bond and particle.resources > bond_threshold and 
                            other.resources > bond_threshold and other.unique_id not in particle.bonds and 
                            np.random.random() < bond_prob):
                        particle.bonds.add(other.unique_id)
                        other.bonds.add(particle.unique_id)
                        particle.resources -= bond_threshold
                        other.resources -= bond_threshold
                        self.bond_events += 1
                        break
                elif particle.is_cell and other.is_cell:
                    if (particle.resources > 10 and other.resources > 10 and 
                            other.unique_id not in particle.bonds and len(particle.bonds) < 3 and 
                            np.random.random() < 0.5):  # Max 2 bonds, 50% chance
                        particle.bonds.add(other.unique_id)
                        other.bonds.add(particle.unique_id)
                        cluster_id = particle.cluster_id or other.cluster_id or self.next_cluster_id
                        if cluster_id == self.next_cluster_id:
                            self.next_cluster_id += 1
                        particle.cluster_id = cluster_id
                        other.cluster_id = cluster_id
                        self.bond_events += 1
                        break
                elif particle.resources > 10 and other.resources < 3 and not other.is_cell:
                    particle.resources += other.resources
                    particle.mass += other.mass * 0.5
                    self.removed_particles.add(other.unique_id)
                    break
        
        # Cell formation: whole chain must qualify
        visited = set()
        for particle in self.particles:
            if particle.unique_id in visited or particle.is_cell:
                continue
            chain = self.get_chain(particle)
            if len(chain) >= 10 and sum(p.resources for p in chain) > 100:
                for p in chain:
                    p.is_cell = True
                    p.radius = 2.0
                    p.base_type = None
                    p.mass = 2.0
            visited.update(p.unique_id for p in chain)
        
        if self.removed_particles:
            for p in self.particles:
                p.bonds -= self.removed_particles
            self.particles = [p for p in self.particles if p.unique_id not in self.removed_particles]
            self.removed_particles.clear()
        
        clusters = {}
        for p in self.particles:
            if p.cluster_id is not None:
                clusters.setdefault(p.cluster_id, []).append(p)
        for cid, members in clusters.items():
            if sum(p.resources for p in members) / len(members) < 10 or len(members) > 10:
                for p in members:
                    p.bonds.clear()
                    p.cluster_id = None

    def get_chain(self, particle):
        visited = set()
        chain = []
        queue = deque([particle.unique_id])
        visited.add(particle.unique_id)
        while queue:
            current_id = queue.popleft()
            current = next(p for p in self.particles if p.unique_id == current_id)
            chain.append(current)
            for bond_id in current.bonds:
                if bond_id not in visited:
                    visited.add(bond_id)
                    queue.append(bond_id)
        return chain

def analyze_chains_and_clusters(particles):
    chain_lengths = []
    cluster_sizes = {}
    visited = set()
    for p in particles:
        if p.unique_id not in visited and p.bonds:
            chain = []
            queue = deque([p.unique_id])
            visited.add(p.unique_id)
            while queue:
                current_id = queue.popleft()
                chain.append(current_id)
                current = next((p for p in particles if p.unique_id == current_id), None)
                if current:
                    for bond_id in current.bonds:
                        if bond_id not in visited:
                            visited.add(bond_id)
                            queue.append(bond_id)
            if current.is_cell and current.cluster_id is not None:
                cluster_sizes[current.cluster_id] = cluster_sizes.get(current.cluster_id, 0) + len(chain)
            else:
                chain_lengths.append(len(chain))
    return chain_lengths, list(cluster_sizes.values())

def run_simulation(gravity, label, steps=2000, runs=5):
    all_stats = []
    for run in range(runs):
        model = LifeModel(200, gravity)
        for step in range(steps + 1):
            model.step()
        chain_lengths, cluster_sizes = analyze_chains_and_clusters(model.particles)
        stats = {
            "bond_events": model.bond_events,
            "population": len(model.particles),
            "cells": sum(1 for p in model.particles if p.is_cell),
            "clusters": len(cluster_sizes),
            "avg_resources": np.mean([p.resources for p in model.particles]) if model.particles else 0,
            "avg_mass": np.mean([p.mass for p in model.particles]) if model.particles else 0,
            "avg_chain_length": np.mean(chain_lengths) if chain_lengths else 0,
            "max_chain_length": max(chain_lengths) if chain_lengths else 0,
            "avg_cluster_size": np.mean(cluster_sizes) if cluster_sizes else 0,
            "max_cluster_size": max(cluster_sizes) if cluster_sizes else 0
        }
        all_stats.append(stats)
    
    avg_stats = {key: np.mean([s[key] for s in all_stats]) for key in all_stats[0].keys()}
    print(f"{label} (Averaged over {runs} runs):")
    print(f"  Bond Events: {avg_stats['bond_events']:.0f}")
    print(f"  Population: {avg_stats['population']:.0f}")
    print(f"  Cells: {avg_stats['cells']:.0f}")
    print(f"  Clusters: {avg_stats['clusters']:.0f}")
    print(f"  Avg Resources: {avg_stats['avg_resources']:.2f}")
    print(f"  Avg Mass: {avg_stats['avg_mass']:.2f}")
    print(f"  Avg Chain Length: {avg_stats['avg_chain_length']:.2f}")
    print(f"  Max Chain Length: {avg_stats['max_chain_length']:.0f}")
    print(f"  Avg Cluster Size: {avg_stats['avg_cluster_size']:.2f}")
    print(f"  Max Cluster Size: {avg_stats['max_cluster_size']:.0f}")
    return avg_stats

print("Normal Gravity")
normal_results = run_simulation(G_NORMAL, "Normal Gravity")
print("\nMicrogravity")
micro_results = run_simulation(G_MICRO, "Microgravity")

Normal Gravity
Normal Gravity (Averaged over 5 runs):
  Bond Events: 396
  Population: 200
  Cells: 153
  Clusters: 1
  Avg Resources: 49.23
  Avg Mass: 1.75
  Avg Chain Length: 20.75
  Max Chain Length: 123
  Avg Cluster Size: 16.00
  Max Cluster Size: 31

Microgravity
Microgravity (Averaged over 5 runs):
  Bond Events: 159
  Population: 200
  Cells: 76
  Clusters: 1
  Avg Resources: 49.60
  Avg Mass: 1.36
  Avg Chain Length: 5.41
  Max Chain Length: 30
  Avg Cluster Size: 39.80
  Max Cluster Size: 41
